In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pyblp as blp
import torch
from torch.autograd import Variable
import torch.optim as optim

blp.options.digits = 2
blp.options.verbose = False
nax = np.newaxis

The file `ps1_ex4.csv` contains aggregate data on $T=600$ markets in which $J=6$ products compete between each other together with an outside good $j=0$. The utility of consumer $i$ is given by:
$$
\begin{aligned}
&u_{i j t} \; = \; \widetilde{\mathbf{x}}_{j t}^{\prime} \boldsymbol{\beta} + \xi_{j t}+\widetilde{\mathbf{x}}_{j t}^{\prime} \boldsymbol{\Gamma} \boldsymbol{v}_{i}+\epsilon_{i j t} \quad j=1, \ldots, 6 \\
&u_{i 0 t} \; = \; \epsilon_{i 0 t}
\end{aligned}
$$
where $x_{j t}$ is a vector of observed product characteristics including the price, $\xi_{j t}$ is an unobserved product characteristic, $v_{i}$ is a vector of unobserved taste shocks for the product characteristics and $\epsilon_{i j t}$ is i.i.d T1EV $(0,1)$. Our goal is to to estimate demand parameters $(\boldsymbol{\beta}, \boldsymbol{\Gamma})$ using the BLP algorithm. As you can see from the data there are only two characteristics $\widetilde{\mathbf{x}}_{j t}=\begin{pmatrix} p_{j t} & x_{j t} \end{pmatrix}$, namely prices and an observed measure of product quality. Moreover, there are several valid instruments $\mathbf{z}_{j t}$ that you will use to construct moments to estimate $(\boldsymbol{\beta}, \boldsymbol{\Gamma})$. Finally, you can assume that $\Gamma$ is lower triangular e.g.,
$$
\boldsymbol{\Gamma} \; = \; \begin{pmatrix}
\gamma_{11} & 0 \\
\gamma_{21} & \gamma_{22}
\end{pmatrix}
$$
such that $\boldsymbol{\Gamma} \boldsymbol{\Gamma}^{\prime}=\boldsymbol{\Omega}$ is a positive definite matrix and that $v_{i}$ is a 2 dimensional vector of i.i.d random taste shocks distributed $\mathcal{N}\left(\mathbf{0}, \mathbf{I}_2 \right)$.

In [4]:
# Load the dataset.
data_ex4 = pd.read_csv('ps1_ex4.csv')
num_prod = data_ex4.choice.max()
num_T = data_ex4.market.max()

# Create outside option shares and merge into dataset.
share_total = data_ex4.groupby(['market'])['shares'].sum().reset_index()
share_total.rename(columns={'shares': 's0'}, inplace=True)
share_total['s0'] = 1 - share_total['s0']
data_ex4 = pd.merge(data_ex4, share_total, on='market')

# Create natural log of share ratios
data_ex4['sr'] = np.log(data_ex4['shares']/data_ex4['s0'])